Bibliotecas

In [ ]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import os
from google.oauth2 import service_account 

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'G:\Vscode projetos\steamdb\Segurança\sdb.json'

client = bigquery.Client()

def carregandoDados(query):
    query_job = client.query(query)
    df = query_job.to_dataframe()
    return df


Subindo dados

In [ ]:
#C_li
# Defina o ID do dataset e da tabela
dataset_id = 'base'
table_id = 'reviews'

# Defina o esquema da tabela
schema = [
    bigquery.SchemaField("app_id", "INTEGER"),
    bigquery.SchemaField("Nome", "STRING"),
    bigquery.SchemaField("positive_reviews", "INTEGER"),
    bigquery.SchemaField("negative_reviews", "INTEGER"),
    bigquery.SchemaField("percentage_reviews", "FLOAT"),
]

# Configuração do job de carregamento
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter=',',
    encoding='UTF-8'
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)

# Caminho do arquivo CSV
file_path = r'G:\Vscode projetos\steamdb\coleta_de_dados\c_li.csv'

# Referencie o dataset e a tabela
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

# Carregue o arquivo CSV
with open(file_path, 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

# Espere o job completar
load_job.result()

print(f'Os dados foram carregados para {table_id}.')

In [ ]:
#C_tr
dataset_id = 'base'
table_id = 'information'

# Defina o esquema da tabela
schema = [
    bigquery.SchemaField("app_id", "INTEGER"),
    bigquery.SchemaField("app_type", "STRING"),
    bigquery.SchemaField("developer", "STRING"),
    bigquery.SchemaField("publisher", "STRING"),
    bigquery.SchemaField("release_date", "STRING")
]

# Configuração do job de carregamento
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter=',',
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)

# Caminho do arquivo CSV
file_path = r'G:\Vscode projetos\steamdb\coleta_de_dados\c_tr.csv'

# Referencie o dataset e a tabela
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

# Carregue o arquivo CSV
with open(file_path, 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

# Espere o job completar
load_job.result()

print(f'Os dados foram carregados para {table_id}.')

In [ ]:
#c_span
dataset_id = 'base'
table_id = 'generos'

# Defina o esquema da tabela
schema = [
    bigquery.SchemaField("app_id", "INTEGER"),
    bigquery.SchemaField("genres", "STRING"),
    bigquery.SchemaField("Nome", "STRING")
]

# Configuração do job de carregamento
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter=',',
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)

# Caminho do arquivo CSV
file_path = r'G:\Vscode projetos\steamdb\coleta_de_dados\c_span.csv'

# Referencie o dataset e a tabela
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

# Carregue o arquivo CSV
with open(file_path, 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

# Espere o job completar
load_job.result()

print(f'Os dados foram carregados para {table_id}.')

In [ ]:
#c_price
dataset_id = 'base'
table_id = 'price'

# Defina o esquema da tabela
schema = [
    bigquery.SchemaField("app_id", "INTEGER"),
    bigquery.SchemaField("price", "FLOAT"),
]

# Configuração do job de carregamento
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    field_delimiter=',',
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)

# Caminho do arquivo CSV
file_path = r'G:\Vscode projetos\steamdb\coleta_de_dados\c_price.csv'

# Referencie o dataset e a tabela
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

# Carregue o arquivo CSV
with open(file_path, 'rb') as source_file:
    load_job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

# Espere o job completar
load_job.result()

print(f'Os dados foram carregados para {table_id}.')

Consulta tabelas

Select

In [ ]:
# Select
extracao = '''
    SELECT
    bi.app_id,
    br.Nome,
    CASE
    WHEN CAST(bp.price AS NUMERIC) IS NULL OR CAST(bp.price AS NUMERIC) = 0 THEN 'Gratuito'
    ELSE FORMAT('%.2f', CAST(bp.price AS NUMERIC) / 100)
    END as price,  
    bi.app_type,
    bg.genres,
    bi.developer,
    bi.publisher,
    br.positive_reviews,
    br.negative_reviews,
    br.percentage_reviews,
    Case
    WHEN REGEXP_CONTAINS(release_date, r'^\d{1,2} \w+ \d{4}$') THEN 
    FORMAT_DATETIME('%m/%d/%Y', PARSE_DATETIME('%d %B %Y', release_date))

    WHEN REGEXP_CONTAINS(release_date, r'^\w+ \d{4}$') THEN 
    FORMAT_DATETIME('%m/%Y', PARSE_DATETIME('%B %Y', release_date))

    WHEN REGEXP_CONTAINS(release_date, r'^\d{4}$') THEN 
    release_date

    ELSE NULL
    END as release_data,
    FROM
    `base.information` as bi
    LEFT JOIN `base.reviews` as br ON bi.app_id = br.app_id
    LEFT JOIN `base.generos` as bg ON bi.app_id = bg.app_id
    LEFT JOIN `base.price` as bp ON bi.app_id = bp.app_id
    '''

In [ ]:
df_tabela_extracao  = carregandoDados(extracao)


Criação de tabelas

In [37]:
df_tabela_extracao.to_csv ('extracaoBQ.csv',  index=False)